In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data

/content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data


In [4]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [5]:
test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
index,,,,,,,,,,,,
0,9.0,0.31,0.48,6.60,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.90,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.20,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.80,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.00,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white
...,...,...,...,...,...,...,...,...,...,...,...,...
995,7.1,0.59,0.02,2.30,0.082,24.0,94.0,0.99744,3.55,0.53,9.7,red
996,8.7,0.15,0.30,1.60,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,white
997,8.8,0.66,0.26,1.70,0.074,4.0,23.0,0.99710,3.15,0.74,9.2,red


# 전처리

## label_encoder

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(train['type'])
train['type'] = label_encoder.transform(train['type'])
test['type'] = label_encoder.transform(test['type'])

## upsampling

In [8]:
from imblearn.over_sampling import SMOTE

In [9]:
X = train.drop('quality', axis=1)  # 입력 변수
y = train['quality']  # 타겟 변수

smote = SMOTE(k_neighbors=4)
X_resampled, y_resampled = smote.fit_resample(X, y)


train_resampled = pd.concat([X_resampled, y_resampled], axis=1)
train_resampled

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,5.600000,0.695000,0.060000,6.800000,0.042000,9.000000,84.000000,0.994320,3.440000,0.440000,10.200000,1,5
1,8.800000,0.610000,0.140000,2.400000,0.067000,10.000000,42.000000,0.996900,3.190000,0.590000,9.500000,0,5
2,7.900000,0.210000,0.390000,2.000000,0.057000,21.000000,138.000000,0.991760,3.050000,0.520000,10.900000,1,5
3,7.000000,0.210000,0.310000,6.000000,0.046000,29.000000,108.000000,0.993900,3.260000,0.500000,10.800000,1,6
4,7.800000,0.400000,0.260000,9.500000,0.059000,32.000000,178.000000,0.995500,3.040000,0.430000,10.900000,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16907,6.834280,0.360000,0.329047,3.630428,0.018657,49.770820,111.551754,0.989767,3.308479,0.414767,12.634280,1,9
16908,7.117607,0.258826,0.482370,2.188262,0.031941,30.765234,114.525982,0.990315,3.364718,0.423521,12.876523,1,9
16909,8.526237,0.267131,0.461475,8.190196,0.034139,28.860644,120.844304,0.995078,3.248770,0.448525,11.117204,1,9
16910,8.429843,0.294126,0.407110,8.187434,0.031247,26.927748,113.545546,0.995030,3.256293,0.500209,10.936126,1,9


# 모델링

## randomforest

In [10]:
y = train_resampled['quality'] #퀄리티 예측해야할거
X = train_resampled.drop('quality', axis=1) #예측해야할거빼고 나머지

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [12]:
model = RandomForestClassifier()

stratified_split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# cv_accuracy = []

n_iter = 0
for train_idx, test_idx in stratified_split.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

    #모델 학습
    model.fit(X_train, y_train)

y_pred=model.predict(test)

# 제출파일

In [13]:
submission = pd.read_csv('sample_submission.csv')

In [14]:
submission

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
995,995,0
996,996,0
997,997,0
998,998,0


In [15]:
submission['quality'] = y_pred

In [16]:
submission.to_csv('upsampling_randomforest.csv', index=False)